In [ ]:
# If an import error occurs try: 
# pip install attrs==19.1.0
# in the cmd
from pathlib import Path
import numpy as np

import pylab as pl
import seaborn as sns
import matplotlib.pyplot as plt

from numpy.testing.decorators import skipif
import mvpa2.suite as mvpa
from mvpa2.measures import rsa

import utility_functions as uf

In [ ]:
# Wider Output
np.set_printoptions(edgeitems=20)

# Project Directory
project_dir = Path.cwd().parents[0]

# BOLD Files Directory
data_dir = project_dir.joinpath("data", "studyforrest-data-aligned")

# Anno Files Directory
anno_dir = project_dir.joinpath("data", "studyforrest-data-annotations")

# Template Directory
template_dir = project_dir.joinpath("data", "studyforrest-data-templatetransforms")

# Mask Directory
mask_dir = project_dir.joinpath("data", "tmp", "masks")
roi_mask_dir = project_dir.joinpath("data", "tmp", "masks", "roi_masks")

# Get Participant Files
ao_files = uf.get_files(data_dir, '*aomovie*nii.gz')
av_files = uf.get_files(data_dir, '*avmovie*nii.gz')

# Get ROI Files
roi_masks_list = uf.get_files(roi_mask_dir, '*.nii.gz')

# Get Annotation Files
new_anno_dir = project_dir.joinpath("data", "tmp", "speech_anno")
ao_anno_files = uf.get_files(new_anno_dir.joinpath("aomovie"), '*.tsv')
av_anno_files = uf.get_files(new_anno_dir.joinpath("avmovie"), '*.tsv')

# Reference Space
ref = template_dir.joinpath("templates", "grpbold3Tp2", "brain.nii.gz")

# Warp Files
warp_files = uf.get_files(template_dir, '*subj2tmpl_warp*.nii.gz')

# Check if all components are good to go
path_list = [project_dir, data_dir, anno_dir, template_dir, ref]
file_lists = [ao_files, av_files, ao_anno_files, av_anno_files, warp_files, roi_masks_list]

check = uf.check_all_components(path_list, file_lists)
print "{}\n".format(check[0])
print "{}\n".format(check[1])

In [ ]:
# DS
ds_save_p = project_dir.joinpath("data", "tmp","preprocessed_ds.hdf5")

# Maske schließt noch zu viel ein (combi aus allen roi masken besser)
mask = project_dir.joinpath("data", "tmp", "masks", "gm_mni_nilearn_mask_adjusted.nii.gz")

targets = ['FORREST', 'MRS. GUMP', 'FORREST (V.O.)', 'LT. DAN']
roi_info = uf.prepare_roi_info(roi_masks_list)

# each run for each participant will be preprocessed individually
ds = uf.load_create_save_ds(ds_save_p, av_files[0:16], ref_space=ref, warp_files=warp_files,
                            mask=mask, detrending="polynomial", use_zscore=True, use_events=True,
                            anno_dir=new_anno_dir, use_glm_estimates=False, targets=targets,
                            event_offset=0, event_dur=3, rois=roi_info)

print "Shape of the dataset: {}\n".format(ds.shape)

print "Mask used: \n{}\n".format(mask)

all_events = ds.sa.targets
unique_events = np.unique(all_events)
num_events = len(ds.sa.targets)

print "Included unique events: {}".format(unique_events)
print "Number of included events: {}\n".format(num_events)

print ds.a
print ds.fa
print ds.sa